In [1]:
#Library Imports
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
import pandas as pd
from sqlalchemy import create_engine

#Database Connection Setup
username = 'postgres'
password = 'postgres'
database = 'vesselsBook'
host = '172.28.37.197:5432'
database_url = f'postgresql://{username}:{password}@{host}/{database}'
engine = create_engine(database_url)

In [2]:
query = '''
SELECT MMSI, t AS Timestamp, SOG, COG, Heading
FROM AISInputSample
WHERE MMSI IN  (246541000, 636018799,311001076,304111000, 211269660, 219014579,219019011,259896000)  
ORDER BY MMSI, t;
'''
df = pd.read_sql_query(query, engine)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df

,mmsi,timestamp,sog,cog,heading
0,211269660,2024-03-01 10:25:45,25.2,270.7,270
1,211269660,2024-03-01 10:26:05,25.3,270.7,271
2,211269660,2024-03-01 10:28:05,25.4,261.4,262
3,211269660,2024-03-01 10:29:38,25.5,262.0,261
4,211269660,2024-03-01 10:30:07,25.5,262.6,261
...,...,...,...,...,...
9677,636018799,2024-03-01 10:59:50,12.6,35.4,40
9678,636018799,2024-03-01 10:59:50,12.6,35.4,40
9679,636018799,2024-03-01 10:59:55,12.6,36.4,41
9680,636018799,2024-03-01 10:59:55,12.6,36.4,41


In [3]:
app = dash.Dash(__name__)
app.layout = html.Div([
  dcc.Dropdown(
    id='mmsi-dropdown',
    options=[{'label': i, 'value': i} for i in df['mmsi'].unique()],
    value=df['mmsi'].unique()[0] ),
    dcc.Graph(id='time-series-plot') ])

In [4]:
@app.callback(
  Output('time-series-plot', 'figure'),
  [Input('mmsi-dropdown', 'value')] )
def update_graph(selected_mmsi):
  filtered_df = df[df['mmsi'] == selected_mmsi]
  scaled_sog = filtered_df['sog'] * 5
  return {
    'data': [
      go.Scatter(
        x=filtered_df['timestamp'], y=filtered_df['sog'],
          mode='lines', name='Scaled SOG (x5)' ),
      go.Scatter(
        x=filtered_df['timestamp'], y=filtered_df['cog'],
          mode='lines', name='COG' ),
      go.Scatter(
        x=filtered_df['timestamp'], y=filtered_df['heading'],
          mode='lines', name='Heading' ) ],
    'layout': go.Layout(
      title='Time Series for MMSI: {}'.format(selected_mmsi),
      xaxis_title='Timestamp',
      yaxis_title='Value',
      margin={'l': 40, 'b': 40, 't': 50, 'r': 10},
      hovermode='closest',
      transition={'duration': 500} ) }

In [5]:
if __name__ == '__main__':
  app.run_server(debug=True)

In [6]:
# Define the window size for smoothing
window_size = 10  # Higer values result in a smoother singal
# Apply rolling mean
df['sog_mean_smoothed'] = df['sog'].rolling(window=window_size, center=True).mean()
df['sog_median_smoothed'] = df['sog'].rolling(window=window_size, center=True).median()
df['cog_mean_smoothed'] = df['cog'].rolling(window=window_size, center=True).mean()
df['cog_median_smoothed'] = df['cog'].rolling(window=window_size, center=True).median()
df['heading_mean_smoothed'] = df['heading'].rolling(window=window_size, center=True).mean()
df['heading_median_smoothed'] = df['heading'].rolling(window=window_size, center=True).median()


In [7]:
app = dash.Dash(__name__)
# App layout
app.layout = html.Div([
dcc.Dropdown(
  id='mmsi-dropdown',
  options=[{'label': i, 'value': i} for i in df['mmsi'].unique()],
   value=df['mmsi'].unique()[0] ),
dcc.Graph(id='time-series-plot') ])

In [8]:
# Initialize Dash app\n",
app = dash.Dash(__name__)
# App layout
app.layout = html.Div([
html.Div([
  dcc.Dropdown(
   id='mmsi-dropdown',
   options=[{'label': i, 'value': i} for i in df['mmsi'].unique()],
   value=df['mmsi'].unique()[0],
  style={'width': '48%', 'display': 'inline-block'}\
  ),
  dcc.Dropdown(
     id='signal-dropdown',
      options=[
      {'label': 'SOG', 'value': 'sog'},
      {'label': 'COG', 'value': 'cog'},
      {'label': 'Heading', 'value': 'heading'}
       ],
      value='sog',
      style={'width': '48%', 'float': 'right', 'display': 'inline-block'}
       ),
      ]),
    dcc.Graph(id='time-series-plot')
    ])

In [9]:
@app.callback(
    Output('time-series-plot', 'figure'),
    [Input('mmsi-dropdown', 'value'), Input('signal-dropdown', 'value')]
    )
def update_graph(selected_mmsi, selected_signal):
 filtered_df = df[df['mmsi'] == selected_mmsi]
# Assuming 'mean_smoothed' and 'median_smoothed' suffixes for your smoothed data columns
 mean_col = f'{selected_signal}_mean_smoothed'
 median_col = f'{selected_signal}_median_smoothed'
 return {
'data': [
 go.Scatter(
     x=filtered_df['timestamp'],
     y=filtered_df[selected_signal],
     mode='lines',
     name=selected_signal
  ),
   go.Scatter(
    x=filtered_df['timestamp'],
    y=filtered_df[mean_col],
    mode='lines',
    name=f'{selected_signal} Mean Smoothed'
    ),
    go.Scatter(
    x=filtered_df['timestamp'],
    y=filtered_df[median_col],
    mode='lines',
    name=f'{selected_signal} Median Smoothed'
    )
    ],
    'layout': go.Layout(
      xaxis_title='Timestamp',
      yaxis_title=selected_signal,
      margin={'l': 80, 'b': 140, 't': 50, 'r': 10},
     font=dict(
     family="Times New Roman",
     size=18,
     color= "black"
     ),
    autosize=False,
    width=1000,
    height=400,
    hovermode='closest') 
 }
    

In [10]:
if __name__ == '__main__':
  app.run_server(debug=True, port=8051)